In [1]:
from pymilvus import model
from pymilvus import MilvusClient, DataType

### Create Milvus connection to a file

In [2]:
client = MilvusClient("/tmp/milvus_demo.db")

client.drop_collection(collection_name="my_sparse_collection")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=False, max_length=100)
schema.add_field(field_name="embeddings", datatype=DataType.SPARSE_FLOAT_VECTOR)

index_params = client.prepare_index_params()

index_params.add_index(field_name="embeddings",
                               index_name="sparse_inverted_index",
                               index_type="SPARSE_INVERTED_INDEX",
                               metric_type="IP",
                               params={"drop_ratio_build": 0.2})
client.create_collection(
    collection_name="my_sparse_collection",
    schema=schema,
    index_params=index_params
)


In [3]:
embeddings_model = model.sparse.SpladeEmbeddingFunction(
    model_name="ibm-granite/granite-embedding-30m-sparse",
    device="cpu",
    batch_size=2,
    k_tokens_query=50,
    k_tokens_document=192
)

In [4]:
import os
os.getenv('CUDA_VISIBLE_DEVICES')

### Prepare documents to be ingested

In [5]:
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
vecs = embeddings_model.encode_documents(docs)

In [6]:
vecs.shape[1]

50265

In [ ]:
dim = vecs.shape[1]
doc_vector = [{"embeddings": doc_emb.reshape(1,dim), "id": f"item_{i}"} for i, doc_emb in enumerate(vecs)]

client.insert(
    collection_name="my_sparse_collection",
    data=doc_vector
)

In [13]:
import torch
tt=vecs[0].to_dense()
print(tt)

AttributeError: 'coo_array' object has no attribute 'to_dense'

### Prepare search parameters

In [8]:
search_params = {
    "params": {"drop_ratio_search": 0.2},  # Additional optional search parameters
}


### Prepare the query vector

In [9]:

queries = [
      "When was artificial intelligence founded",
      "Where was Turing born?"
]
query_vector = embeddings_model.encode_documents(queries)

res = client.search(
    collection_name="my_sparse_collection",
    data=query_vector,
    limit=1, #top k documents to return
    output_fields=["id"],
    search_params=search_params,
)

for r in res:
    print(r)

[{'id': '456330090434789376', 'distance': 12.364131927490234, 'entity': {'id': 'item_0'}}]
[{'id': '456330090434789378', 'distance': 17.135875701904297, 'entity': {'id': 'item_2'}}]
